---

### **Import the SDK via NUGET**

In [ ]:
#r "nuget: Microsoft.SemanticKernel, 0.17.230704.3-preview"
//#r "nuget: Microsoft.SemanticKernel.Skills.OpenAPI, 0.17.230704.3-preview"

//#r "nuget: Microsoft.SemanticKernel, 0.17.230626.1-preview"




---
### **Load configuration from secrets.json**

In [ ]:
#!import ../Settings/AppConfig.cs

//#!import /Settings/AppConfig.cs

AppConfig.aoaiSettings apiConfig = AppConfig.GetSecrets();

// Get AOAI values
//string mydeploymentName = apiConfig.SKdeploymentName; // davinci
string mydeploymentName = apiConfig.deploymentName; // turbo
string myendpoint = apiConfig.endpoint;
string myAPIKey = apiConfig.APIKey;

//NOTE: If you run the below writeline command, be sure to clear the output before pushing to repo!!
//Console.WriteLine(mydeploymentName + " & " + myendpoint + " & " + myAPIKey);


---
### **Build the Semantic Kernel client**

In [ ]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Orchestration;
using Microsoft.SemanticKernel.AI.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.AI.OpenAI.ChatCompletion;
using Microsoft.SemanticKernel.SkillDefinition;
//using Microsoft.SemanticKernel.Skills.OpenAPI.Extensions;
//using Microsoft.SemanticKernel.Skills.OpenAPI.Skills;

using System;



 IKernel kernel = new KernelBuilder()            
 //var kernel = new KernelBuilder()                      
            // Note: we use Chat Completion and GPT 3.5 Turbo
            .WithAzureChatCompletionService(mydeploymentName, myendpoint, myAPIKey)
            .Build();

// Load the Skills Directory
var skillsDirectory = Path.Combine(System.IO.Directory.GetCurrentDirectory(), "SKSkills");
// Load the Chat function from the FunSkill
var skills = kernel.ImportSemanticSkillFromDirectory(skillsDirectory, "ChatSkills");

 
//var skill3 = await kernel.ImportChatGptPluginSkillSkillFromDirectoryAsync(skillsDirectory, "ChatSkills");

//var skill2 =  kernel.ImportChatGptPluginSkillSkillFromFile();

//kernel.ImportChatGptPluginSkillFromUrlAsync
//var skill = await kernel.ImportChatGptPluginSkillFromUrlAsync("<skill name>", new Uri("<chatGPT-plugin>"), new OpenApiSkillExecutionParameters(httpClient));


AzureChatCompletion chatGPT = new(mydeploymentName, myendpoint, myAPIKey);

var dSkill = skills["BatmanChat"];
Console.WriteLine(dSkill.ToString());
//dSkill.



var test = skills["BatmanChat"].ToString();
Console.WriteLine(test);
//var chatHistory = chatGPT.CreateNewChat(skills["BatmanChat"]);
var chatHistory = chatGPT.CreateNewChat(skills["BatmanChat"]);
//var chatHistory = chatGPT.CreateNewChat(skills["BatmanChat"].ToString());

Console.WriteLine(chatHistory);
 // First bot assistant message
        string reply = await chatGPT.GenerateMessageAsync(chatHistory);
        chatHistory.AddAssistantMessage(reply);
        Console.WriteLine(chatHistory);





---
### **Simple chat loop**

In [ ]:
using Microsoft.SemanticKernel.Orchestration;
using InteractiveKernel = Microsoft.DotNet.Interactive.Kernel;

// Load the Skills Directory
var skillsDirectory = Path.Combine(System.IO.Directory.GetCurrentDirectory(), "SKSkills");

// Load the Chat function from the FunSkill
var skill = kernel.ImportSemanticSkillFromDirectory(skillsDirectory, "ChatSkill");

var myContext = new ContextVariables(); 
var botPrompt = "AI: Hello. What's your name?";
var history = $"{botPrompt}\n";

const int numberOfRounds = 4;

myContext.Set("history", history); 

for(var i = 0; i < numberOfRounds; i++) {
    try {
        // get input from the user and set the context variable        
        var input = await InteractiveKernel.GetInputAsync($"{botPrompt} ({(i+1)} of {numberOfRounds})");
        myContext.Set("input", input); 

        // run the chat function
        var myResult = await kernel.RunAsync(myContext,skill["Chat"]);

        // tack onto the history what's come back from the model
        /********************************************************/
        var theNewChatExchange = $"Me: {input}\nAI:{myResult}\n";
        history += theNewChatExchange;
        myContext.Set("history", history); 
        /********************************************************/
        // this way the new chat exchange gets passed into the next round

        // announce the number of rounds and the history
        Console.WriteLine($"Chat for {i+1} of {numberOfRounds} rounds with AI:\n{history}");

        // prepare to "prompt" the user with the bot's response
        botPrompt = $"AI: {myResult}";
    } catch {
        // if the user hits "Escape" we end the chat early
        Console.WriteLine("AI: Thanks for the wonderful chat!");
        break;
    }
}

---
### **Batman Sample Chat**

In [ ]:
using Microsoft.SemanticKernel.Orchestration;
using InteractiveKernel = Microsoft.DotNet.Interactive.Kernel;

// Load the Skills Directory
var skillsDirectory = Path.Combine(System.IO.Directory.GetCurrentDirectory(), "SKSkills");

// Load the Chat function from the 
var skill = kernel.ImportSemanticSkillFromDirectory(skillsDirectory, "ChatSkill");

var myContext = new ContextVariables(); 
var botPrompt = "Batman: Hello Batman Fan.  Do you have a question for me?";
var history = $"{botPrompt}\n";

const int numberOfRounds = 4;

myContext.Set("history", history); 

for(var i = 0; i < numberOfRounds; i++) {
    try {
        // get input from the user and set the context variable
        Console.WriteLine("👆 Enter text in the input cell above to chat with the bot. 👆\n");
        var input = await InteractiveKernel.GetInputAsync($"{botPrompt} ({(i+1)} of {numberOfRounds})");
        myContext.Set("input", input); 

        // run the chat function
        var myResult = await kernel.RunAsync(myContext,skill["BatmanChat"]);

        // tack onto the history 👇 what's come back from the model
        /********************************************************/
        var theNewChatExchange = $"Fan: {input}\nAI:{myResult}\n";
        history += theNewChatExchange;
        myContext.Set("history", history); 
        /********************************************************/
        // this way the new chat exchange gets passed into the next round

        // announce the number of rounds and the history
        Console.WriteLine($"Chat for {i+1} of {numberOfRounds} rounds with AI:\n{history}");

        // prepare to "prompt" the user with the bot's response
        botPrompt = $"Batman: {myResult}";
    } catch {
        // if the user hits "Escape" we end the chat early
        Console.WriteLine("Batman: Thanks for the wonderful chat!");
        break;
    }
}